In [1]:
%load_ext autoreload
%autoreload 2


In [50]:
import guidance

# guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo", chat_completion=True)
guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", chat_completion=False)

In [51]:
from bs4 import BeautifulSoup
def extract_stuff(html_page):
    soup = BeautifulSoup(html_page, 'html.parser').find('div', {'class': 'problem-statement'})
    # soup = BeautifulSoup(html_tag, 'html.parser')

    problem_statement = str(soup.find('div', class_=''))
    input_specification = str(soup.find('div', class_='input-specification'))
    output_specification = str(soup.find('div', class_='output-specification'))

    example_input = []
    example_output = []

    for example in soup.find_all('div', class_='sample-test'):
        input_example = '\n'.join([a.text for a in example.find('div', class_='input').find('pre').find_all('div')])
        output_example = example.find('div', class_='output').find('pre').text.strip()
        if not input_example:
            for input_example in example.find_all('div', class_='input'):
                input_example = input_example.find('pre')
                for br in input_example.find_all("br"):
                    br.replace_with("\n")
                input_example = input_example.text.strip()
                example_input.append(input_example)
            for output_example in example.find_all('div', class_='output'):
                output_example = output_example.find('pre')
                for br in output_example.find_all("br"):
                    br.replace_with("\n")
                output_example = output_example.text.strip()
                example_output.append(output_example)
            print(input_example)
            print(output_example)
        else:
            example_input.append(input_example)
            example_output.append(output_example)

    note = str(soup.find('div', class_='note'))

    result = {
        'problem_statement': problem_statement,
        'input_specification': input_specification,
        'output_specification': output_specification,
        'examples': [{'input': x, 'output': y} for x, y in zip(example_input, example_output)],
        # 'example_input': example_input,
        # 'example_output': example_output,
        'note': note
    }
    return result

In [52]:
import requests

url = "https://codeforces.com/contest/1791/problem/C"
# url = 'https://codeforces.com/problemset/problem/1807/D'
# url = 'https://codeforces.com/contest/787/problem/A'
# url = 'https://codeforces.com/contest/793/problem/B'
response = requests.get(url)
stuff = extract_stuff(response.text)
stuff
# response.text

{'problem_statement': '<div><p>Timur initially had a binary string$$$^{\\dagger}$$$ $$$s$$$ (possibly of length $$$0$$$). He performed the following operation several (possibly zero) times: </p><ul> <li> Add $$$\\texttt{0}$$$ to one end of the string and $$$\\texttt{1}$$$ to the other end of the string. For example, starting from the string $$$\\texttt{1011}$$$, you can obtain either $$$\\color{red}{\\texttt{0}}\\texttt{1011}\\color{red}{\\texttt{1}}$$$ or $$$\\color{red}{\\texttt{1}}\\texttt{1011}\\color{red}{\\texttt{0}}$$$. </li></ul> You are given Timur\'s final string. What is the length of the <span class="tex-font-style-bf">shortest</span> possible string he could have started with?<p>$$$^{\\dagger}$$$ A binary string is a string (possibly the empty string) whose characters are either $$$\\texttt{0}$$$ or $$$\\texttt{1}$$$.</p></div>',
 'input_specification': '<div class="input-specification"><div class="section-title">Input</div><p>The first line of the input contains an intege

In [53]:
guidance.llms.OpenAI.cache.clear()


48

## v2 , with CoT split over queries

In [54]:
# New attempt, with different breakdown
conversation_question = guidance('''Solve a codeforce problem.
Here is the problem, in html:
PROBLEM STATEMENT:
{{problem_statement}}
INPUT SPECIFICATION:
{{input_specification}}
OUTPUT SPECIFICATION:
{{output_specification}}
EXAMPLES:
{{#each examples}}INPUT:
```inputs
{{this.input}}
```
OUTPUT:
```output
{{this.output}}
```{{/each}}
NOTE:
{{note}}
---''')
conversation_question = conversation_question(**stuff)
dl = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{{conversation_question}}<|im_end|>
{{#each 'conversation'}}
<|im_start|>user
Please describe the problem in your own words, and explain how a few of the example inputs lead to the example outputs<|im_end|>
<|im_start|>assistant
{{generate 'this.description' stop="<|im_end|>" temperature=0 max_tokens=900}}<|im_end|>
<|im_start|>user
Write python 3.8.10 code to solve the problem. When you start your code, please use ```python as the first line. Your code should be able to read inputs like the original input examples, and output the corresponding outputs. When you end your code, please use ``` as the last line.<|im_end|>
<|im_start|>assistant
{{generate 'this.code' stop="<|im_end|>" temperature=0 max_tokens=900}}<|im_end|>
<|im_start|>user
I am going to run your code. Please write inputs you'd want me to run, using ```inputs in the first line (make sure to include the example inputs, since we have ground truth outputs for those). Also make sure to follow the same format as the example inputs.<|im_end|>
<|im_start|>assistant
{{generate 'this.inputs' stop="<|im_end|>" temperature=0 max_tokens=900}}<|im_end|>
<|im_start|>user
{{set 'this.user_text' (await 'user_text')}}<|im_end|>
<|im_start|>assistant
{{generate 'this.right' stop="<|im_end|>" temperature=0 max_tokens=900}}<|im_end|>
<|im_start|>user
{{set 'this.after' (await 'after')}}<|im_end|>
<|im_start|>assistant
{{generate 'this.probzzz' stop="<|im_end|>" temperature=0 max_tokens=900}}<|im_end|>
<|im_start|>user
Let's try again, then<|im_end|>{{/each}}''', echo=True)
import re
import os
def run_code(conversation):
    code = re.findall(r'```python\n(.*?)```', conversation['code'], re.DOTALL)[-1]
    inputs = re.findall(r'```inputs\n(.*?)```', conversation['inputs'], re.DOTALL)[-1]
    open('/tmp/a.py', 'w').write(code)
    open('/tmp/input', 'w').write(inputs)
    out = os.popen('python /tmp/a.py < /tmp/input').read()
    return out

r = dl(conversation_question=str(conversation_question))
for round in range(3):
    #conversation = r['conversation'][-2]
    conversation = r['conversation'][-1]
    out = run_code(conversation)
    out = f'''Here is the output of your code on the provided inputs
```output
{out}```
Is your code correct? If so, respond with SUBMIT. If not, respond with NO.'''
    r = r(user_text=out)
    #conversation = r['conversation'][-2]
    conversation = r['conversation'][-1]
    last_ai = conversation['right']
    if 'SUBMIT' in last_ai:
        print('SUBMIT')
        break
    r = r(after='''What went wrong? Please describe the problem in your own words, and explain how a few of the example inputs lead to the example outputs''')

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Solve a codeforce problem.
Here is the problem, in html:
PROBLEM STATEMENT:
<div><p>Timur initially had a binary string$$$^{dagger}$$$ $$$s$$$ (possibly of length $$$0$$$). He performed the following operation several (possibly zero) times: </p><ul> <li> Add $$$texttt{0}$$$ to one end of the string and $$$texttt{1}$$$ to the other end of the string. For example, starting from the string $$$texttt{1011}$$$, you can obtain either $$$color{red}{texttt{0}}texttt{1011}color{red}{texttt{1}}$$$ or $$$color{red}{texttt{1}}texttt{1011}color{red}{texttt{0}}$$$. </li></ul> You are given Timur's final string. What is the length of the <span class="tex-font-style-bf">shortest</span> possible string he could have started with?<p>$$$^{dagger}$$$ A binary string is a string (possibly the empty string) whose characters are either $$$texttt{0}$$$ or $$$texttt{1}$$$.</p></div>
INPUT SPECIFICATION:
<div class="input-specification"><div class="section-title">Input</div><p>The first line of the input contains an integer $$$t$$$ ($$$1 leq t leq 100$$$) — the number of testcases.</p><p>The first line of each test case contains an integer $$$n$$$ ($$$1 leq n leq 2000$$$) — the length of Timur's final string.</p><p>The second line of each test case contains a string $$$s$$$ of length $$$n$$$ consisting of characters $$$texttt{0}$$$ or $$$texttt{1}$$$, denoting the final string.</p></div>
OUTPUT SPECIFICATION:
<div class="output-specification"><div class="section-title">Output</div><p>For each test case, output a single nonnegative integer — the shortest possible length of Timur's original string. Note that Timur's original string could have been empty, in which case you should output $$$0$$$.</p></div>
EXAMPLES:
INPUT:
```inputs
9
3
100
4
0111
5
10101
6
101010
7
1010110
1
1
2
10
2
11
10
1011011010
```
OUTPUT:
```output
1
2
5
0
3
1
0
2
4
```
NOTE:
<div class="note"><div class="section-title">Note</div><p>In the first test case, the shortest possible string Timur started with is $$$texttt{0}$$$, and he performed the following operation: $$$texttt{0} to color{red}{texttt{1}}texttt{0}color{red}{texttt{0}}$$$.</p><p>In the second test case, the shortest possible string Timur started with is $$$texttt{11}$$$, and he performed the following operation: $$$texttt{11} to color{red}{texttt{0}}texttt{11}color{red}{texttt{1}}$$$.</p><p>In the third test case, the shortest possible string Timur started with is $$$texttt{10101}$$$, and he didn't perform any operations.</p><p>In the fourth test case, the shortest possible string Timur started with is the empty string (which we denote by $$$varepsilon$$$), and he performed the following operations: $$$varepsilon to color{red}{texttt{1}}texttt{}color{red}{texttt{0}} to color{red}{texttt{0}}texttt{10}color{red}{texttt{1}} to color{red}{texttt{1}}texttt{0101}color{red}{texttt{0}}$$$.</p><p>In the fifth test case, the shortest possible string Timur started with is $$$texttt{101}$$$, and he performed the following operations: $$$texttt{101} to color{red}{texttt{0}}texttt{101}color{red}{texttt{1}} to color{red}{texttt{1}}texttt{01011}color{red}{texttt{0}}$$$.</p></div>
---<|im_end|>

<|im_start|>user
Please describe the problem in your own words, and explain how a few of the example inputs lead to the example outputs<|im_end|>
<|im_start|>assistant
The problem is about finding the shortest possible binary string that Timur could have started with, given his final binary string. Timur can perform an operation where he adds a '0' to one end of the string and a '1' to the other end of the string. The input consists of multiple test cases, and for each test case, we need to find the shortest possible length of Timur's original string.

Let's go through a few examples:

Example 1:
Input: 3 (length of final string), 100 (final string)
Output: 1
Explanation: The shortest possible string Timur started with is '0', and he performed the following operation: 0 -

In [48]:
r['conversation']

[{'description': 'The problem is to determine the shortest possible binary string that Timur could have started with, given the final binary string he ended up with. To do this, we need to consider the operations he performed to get from the original string to the final string. \n\nIn the first example, the shortest possible string Timur started with is "0", and he performed the operation of adding a "1" to one end and a "0" to the other end. This resulted in the final string "100". \n\nIn the second example, the shortest possible string Timur started with is "11", and he performed the operation of adding a "0" to one end and a "1" to the other end. This resulted in the final string "0111". \n\nIn the third example, the shortest possible string Timur started with is "10101", and he didn\'t perform any operations. This resulted in the final string "10101". \n\nIn the fourth example, the shortest possible string Timur started with is the empty string, and he performed the operations of a

'NO'

## V1, CoT all at once

In [17]:
conversation_question = guidance('''Solve a codeforce problem.
Here is the problem, in html:
PROBLEM STATEMENT:
{{problem_statement}}
INPUT SPECIFICATION:
{{input_specification}}
OUTPUT SPECIFICATION:
{{output_specification}}
EXAMPLES:
{{#each examples}}INPUT:
```inputs
{{this.input}}
```
OUTPUT:
```output
{{this.output}}
```{{/each}}
NOTE:
{{note}}
---
Now please follow these steps:
1. Describe the problem in your own words, and explain how a few of the example inputs lead to the example outputs, and how the algorithm you'll propose would work on these examples.
2. Write python 3.8.10 code to solve the problem. When you start your code, please use ```python as the first line. Your code should be able to read inputs like the input examples, and output the corresponding outputs. When you end your code, please use ``` as the last line.
3. After you are done, I will run your code, so please also write inputs you'd want me to run, using ```inputs in the first line (make sure to include the example inputs, since we have ground truth outputs for those). Also make sure to follow the same format as the example inputs.''')
conversation_question = conversation_question(**stuff)
dl = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|> {{#each 'conversation'}}
<|im_start|>user
{{set 'this.user_text' (await 'user_text')}}<|im_end|>
<|im_start|>assistant
{{generate 'this.ai_text' temperature=0 max_tokens=900}}<|im_end|>{{/each}}''', echo=True)
import re
import os
def run_code(ai_text):
    code = re.findall(r'```python\n(.*?)```', ai_text, re.DOTALL)[-1]
    inputs = re.findall(r'```inputs\n(.*?)```', ai_text, re.DOTALL)[-1]
    open('/tmp/a.py', 'w').write(code)
    open('/tmp/input', 'w').write(inputs)
    out = os.popen('python /tmp/a.py < /tmp/input').read()
    return out
def user_text(ai_text):
    output = run_code(ai_text)
    return f'''Here is the output of your code on the provided inputs
```output
{output}```
If you think your code is correct, please respond with SUBMIT.
Otherwise, please do the following steps (make sure to list each step in your response):
1. Tell me what was wrong with the previous algorithm or code. Be as specific as possible, explain if it was a problem in the algorithm or in the implementation, and what the problem was.
2. Tell me how you will solve the problem, step by step, and how your new algorithm will work on some of the examples.
3. Propose a new solution and set of inputs for me to run it on.'''

r = dl(user_text=str(conversation_question))
for round in range(10):
    last_ai = r['conversation'][-2]['ai_text']
    print(last_ai)
    if 'SUBMIT' in last_ai:
        print('SUBMIT')
        break
    user_response = user_text(last_ai)
    r = r(user_text=user_response)

<|im_start|>system
You are a helpful assistant.<|im_end|> 
<|im_start|>user
Solve a codeforce problem.
Here is the problem, in html:
PROBLEM STATEMENT:
<div><p>Timur initially had a binary string$$$^

Error in prompt:  This model's maximum context length is 8192 tokens. However, you requested 8215 tokens (7315 in the messages, 900 in the completion). Please reduce the length of the messages or completion.


InvalidRequestError: This model's maximum context length is 8192 tokens. However, you requested 8215 tokens (7315 in the messages, 900 in the completion). Please reduce the length of the messages or completion.

In [8]:
r['conversation']

[{'user_text': 'Solve a codeforce problem.\nHere is the problem, in html:\nPROBLEM STATEMENT:\n<div><p>Timur initially had a binary string$$$^{\\dagger}$$$ $$$s$$$ (possibly of length $$$0$$$). He performed the following operation several (possibly zero) times: </p><ul> <li> Add $$$\\texttt{0}$$$ to one end of the string and $$$\\texttt{1}$$$ to the other end of the string. For example, starting from the string $$$\\texttt{1011}$$$, you can obtain either $$$\\color{red}{\\texttt{0}}\\texttt{1011}\\color{red}{\\texttt{1}}$$$ or $$$\\color{red}{\\texttt{1}}\\texttt{1011}\\color{red}{\\texttt{0}}$$$. </li></ul> You are given Timur\'s final string. What is the length of the <span class="tex-font-style-bf">shortest</span> possible string he could have started with?<p>$$$^{\\dagger}$$$ A binary string is a string (possibly the empty string) whose characters are either $$$\\texttt{0}$$$ or $$$\\texttt{1}$$$.</p></div>\nINPUT SPECIFICATION:\n<div class="input-specification"><div class="section

In [ ]:
print(r['conversation'][1]['ai_text'])
print()
print(r['conversation'][2]['ai_text'])

I apologize for the confusion. I realize that there is a mistake in my code. The issue is with the can_reach function, which checks if two points can be reached in a straight line without encountering road works. The function should return True if the two points are the same, but it currently does not handle this case.

1. The problem with the previous code is in the implementation of the can_reach function. It does not handle the case where the start and end points are the same.

2. To fix the issue, I will modify the can_reach function to return True if the start and end points are the same. This will ensure that the function works correctly for all cases.

Here is the updated code:

```python
def can_reach_office(n, m, grid):
    for i in range(n):
        for j in range(m):
            if grid[i][j] == 'S':
                start = (i, j)
            if grid[i][j] == 'T':
                end = (i, j)

    def can_reach(start, end):
        if start == end:
            return True
  

In [ ]:
code = re.findall(r'```python\n(.*?)```', r['options'], re.DOTALL)[0]
open('/tmp/a.py', 'w').write(code)
open('/tmp/input', 'w').write(stuff['example_input'])
out = os.popen('python /tmp/a.py < /tmp/input').read()
out

# r

'1\n0\n1\n1\n1\n1\n1\n2\n1\n'

In [ ]:
import re

"def shortest_possible_length(s):\n    count = 0\n    for c in s:\n        if c == '1':\n            count += 1\n        else:\n            break\n    return count\n\nt = int(input())\nfor _ in range(t):\n    n = int(input())\n    s = input()\n    result = shortest_possible_length(s)\n    print(result)\n"

In [ ]:
import os
# open('/tmp/a.py', 'w').write(r['options'].strip('```').strip('python\n'))
code = re.findall(r'```python\n(.*?)```', r['options'], re.DOTALL)[0]
open('/tmp/a.py', 'w').write(code)
open('/tmp/input', 'w').write(stuff['example_input'])
out = os.popen('python /tmp/a.py < /tmp/input').read()
# compare this with stuff['example_output']
print(list(zip(out.split('\n'), stuff['example_output'].split('\n'))))

[('1', '1'), ('0', '2'), ('1', '5'), ('1', '0'), ('1', '3'), ('1', '1'), ('1', '0'), ('2', '2'), ('1', '4')]


In [ ]:

print(out)

1
1
3
5
6
0
1
0
9



In [ ]:
def shortest_initial_string(t, test_cases):
    results = []
    for i in range(t):
        n, s = test_cases[i]
        zero_count, one_count = 0, 0
        for char in s:
            if char == '0':
                zero_count += 1
            else:
                one_count += 1
                if zero_count > 0:
                    zero_count -= 1

        results.append(min(zero_count, one_count))

    return results
outs = stuff['example_output'].split('\n')
j = 0 
for i in range(2, len(stuff['example_input'].split('\n')[1:]) + 1, 2):
    iz = stuff['example_input'].split('\n')[i]
    print(iz, outs[j], shortest_initial_string(iz))
    j += 1

TypeError: shortest_initial_string() missing 1 required positional argument: 'test_cases'

In [ ]:
stuff['example_input']
def min_initial_length(s):
    count = 0
    while s:
        if s[0] == s[-1]:
            break
        count += 1
        s = s[1:-1]
    return count
min_initial_length('101010')

3

'3 4 5 '

In [ ]:
def min_initial_length(s):
    count = 0
    while s:
        if s[0] == s[-1]:
            break
        count += 1
        s = s[1:-1]
    return count

min_initial_length(3)

TypeError: 'int' object is not subscriptable

In [ ]:
stuff

{'problem_statement': '<div><p>Timur initially had a binary string$$$^{\\dagger}$$$ $$$s$$$ (possibly of length $$$0$$$). He performed the following operation several (possibly zero) times: </p><ul> <li> Add $$$\\texttt{0}$$$ to one end of the string and $$$\\texttt{1}$$$ to the other end of the string. For example, starting from the string $$$\\texttt{1011}$$$, you can obtain either $$$\\color{red}{\\texttt{0}}\\texttt{1011}\\color{red}{\\texttt{1}}$$$ or $$$\\color{red}{\\texttt{1}}\\texttt{1011}\\color{red}{\\texttt{0}}$$$. </li></ul> You are given Timur\'s final string. What is the length of the <span class="tex-font-style-bf">shortest</span> possible string he could have started with?<p>$$$^{\\dagger}$$$ A binary string is a string (possibly the empty string) whose characters are either $$$\\texttt{0}$$$ or $$$\\texttt{1}$$$.</p></div>',
 'input_specification': '<div class="input-specification"><div class="section-title">Input</div><p>The first line of the input contains an intege